# Fine Tuned Bert

### For Comments

In [ ]:
!pip install -U transformers datasets scikit-learn pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2

In [ ]:
import os
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)

In [ ]:
train = pd.read_csv("https://raw.githubusercontent.com/Fahmi-mi/Dataset/refs/heads/main/datathon-ristek-ui-2025/input_instagram_brands/train_bert_comment.csv")

In [ ]:
os.environ["WANDB_DISABLED"] = "true"
df = train.dropna(subset=["comment", "predicted_label"])


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["comment"].tolist(), df["predicted_label"].tolist(), test_size=0.2, random_state=42
)

label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
label_names = label_encoder.classes_.tolist()

# Tokenisasi
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Dataset PyTorch
class KomentarDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {"labels": torch.tensor(self.labels[idx])}

train_dataset = KomentarDataset(train_encodings, train_labels_encoded)
val_dataset = KomentarDataset(val_encodings, val_labels_encoded)

# Load model dan training setup
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=len(label_names)
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to=[], 
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Mulai training
trainer.train()

# Simpan model & tokenizer
save_dir = "./model_finetuned"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,2.040000
20,1.556400
30,1.303000
40,1.408500
50,1.424300
60,1.213700
70,1.188200
80,1.278800
90,1.287000
100,1.236200


('./model_finetuned/tokenizer_config.json',
 './model_finetuned/special_tokens_map.json',
 './model_finetuned/vocab.txt',
 './model_finetuned/added_tokens.json')

In [ ]:
import json

# Simpan label_names
with open(f"{save_dir}/label_names.json", "w") as f:
    json.dump(label_names, f)

print("✅ label_names berhasil disimpan ke model_finetuned/label_names.json")


✅ label_names berhasil disimpan ke model_finetuned/label_names.json


In [ ]:
!zip -r model_finetuned.zip model_finetuned

updating: model_finetuned/ (stored 0%)
updating: model_finetuned/config.json (deflated 55%)
updating: model_finetuned/special_tokens_map.json (deflated 42%)
updating: model_finetuned/tokenizer_config.json (deflated 75%)
updating: model_finetuned/vocab.txt (deflated 53%)
updating: model_finetuned/model.safetensors (deflated 7%)
  adding: model_finetuned/label_names.json (deflated 32%)


In [ ]:
from huggingface_hub import notebook_login, upload_folder
notebook_login()
upload_folder(
    repo_id="AzrilFahmiardi/instagram-comments-classifier-bert",
    folder_path="./model_finetuned",            
    path_in_repo=".",                           
    commit_message="Upload model komentar (fine-tuned BERT dengan safetensors)"
)

### For Caption

In [ ]:
!pip install -U transformers datasets scikit-learn pandas tqdm

In [ ]:
import os
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)


In [ ]:

os.environ["WANDB_DISABLED"] = "true"
df = pd.read_csv("https://raw.githubusercontent.com/Fahmi-mi/Dataset/refs/heads/main/datathon-ristek-ui-2025/input_instagram_brands/train_bert_caption.csv")
df = df.dropna(subset=["post_caption", "label_1"])

# Split data: train & validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["post_caption"].tolist(), df["label_1"].tolist(), test_size=0.2, random_state=42
)

# Encode label ke angka
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
label_names = label_encoder.classes_.tolist()

# Tokenisasi caption
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Dataset PyTorch
class CaptionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            key: torch.tensor(val[idx]) for key, val in self.encodings.items()
        } | {"labels": torch.tensor(self.labels[idx])}

train_dataset = CaptionDataset(train_encodings, train_labels_encoded)
val_dataset = CaptionDataset(val_encodings, val_labels_encoded)

# Load model BERT dan setup Trainer
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=len(label_names)
)

training_args = TrainingArguments(
    output_dir="./caption_model",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    report_to=[],  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Training model
trainer.train()

# Simpan model, tokenizer, dan label_names
save_dir = "./caption_model"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Simpan daftar nama label
import json
with open(f"{save_dir}/label_names.json", "w") as f:
    json.dump(label_names, f)

print("✅ Model & label disimpan di:", save_dir)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
10,1.508600
20,1.394600
30,1.118200
40,1.127900
50,0.994400
60,0.887600


Step,Training Loss
10,1.508600
20,1.394600
30,1.118200
40,1.127900
50,0.994400
60,0.887600
70,0.810000


✅ Model & label disimpan di: ./caption_model


In [ ]:
from huggingface_hub import notebook_login, upload_folder, create_repo

notebook_login()

upload_folder(
    repo_id="AzrilFahmiardi/instagram-caption-classifier-bert",
    folder_path="./caption_model",
    path_in_repo=".",
    commit_message="Upload model komentar (format .safetensors)"
)